In [55]:
#everthing is found except the Name is not suppporting
import cv2
import pytesseract
import re

# Load the image using OpenCV
image_path = 'bbva_assets/bbva.png'
image = cv2.imread(image_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Use pytesseract to extract text from the grayscale image
extracted_text = pytesseract.image_to_string(gray_image)

# print(extracted_text)

lines = extracted_text.strip().split('\n')

'''
#  line_4_rename = r'\b(?:[A-Za-z,:]+\s+)*([A-Za-z,\.]+)\s+\w+$'
# line_41 = line_4_rename
 
# print(line_41)

# removes 1st word
__line_4_ = re.sub(r'^\w+:+\s+\b', '', line_4)
print(__line_4_)

# removes last word
line_4_ = re.sub(r'\b\s+\w+$', '', __line_4_)
print(line_4_)
'''

#for payer name
line_4 = lines[4]
#removes 1st word
line_4 = re.sub(r'^\w+:+\s+\b', '', line_4)
# print(line_4s)
#removes last word
line_4 = re.sub(r'\b\s+\w+$', '', line_4)
# print(line_4)


#for proof number
line_3 = lines[3]
#removes 1st word
# line_3 = re.sub(r'^\w+:+\s+\b', '', line_3)
# Cuenta_Origen = r'^Cuenta Origen:\s+CC\s+\$\s'
Cuenta_Origen = r'^Cuenta Origen:\s+CC\s+\$\s+\d{4}-\d{6}\/\d{1}\s+'
line_3 = re.sub(Cuenta_Origen, '', line_3)
line_3 = re.sub(r' ' , '', line_3)
# print(line_3)

#for amount
line_12 = lines[12]
# print(line_12)
#removes 1st word
line_12 = re.sub(r'^\w+:+\s+\b', '', line_12)
#adds $ 
line_12 = '$ ' + line_12
# print(line_12)


# Define a regular expression pattern to match a date format like "27 de marzo 2024"
bank_pattern = 'BBVA'
date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'

proof_number_pattern = line_3 #have to ask martin about this
# payer_name_pattern = r'(?:\n)?([A-Z][a-z]+(?: [A-Z][a-z]+)*)'
payer_name_pattern = line_4  
# this bank may be not providing the $ before amount so adding it
amount_pattern = line_12 #this regEx for regular-- r'\b\$?\s?\d{1,3}(?:,\d{3})*\.\d{2}\b

# print(proof_number_pattern,amount_pattern)

cuit_pattern =  r'\b\d{2}-\d{8}-\d{1}\b' #MARTIN SAID Cuit IS FILLED MANUALLY 

# Find all occurrences of dates, amounts, and CUIT numbers in the extracted text
bank_found = re.findall(bank_pattern, extracted_text, re.IGNORECASE)

dates_found = re.findall(date_pattern, extracted_text)

amounts_found = re.findall(amount_pattern, extracted_text)
proof_number_found = re.findall(proof_number_pattern, extracted_text)  # Use cuit_pattern
payer_name_found = re.findall(payer_name_pattern , extracted_text)

cuit_found = re.findall(cuit_pattern, extracted_text)  # Use cuit_pattern

# Extract the first date, amount, and CUIT number if any are found
date = dates_found[0] if dates_found else None
bank = bank_found[0] if bank_found else None


amount = line_12
proof_number = line_3
# payer = payer_name_found[0] if payer_name_found else None #this works
payer = line_4 #this just for testing

cuit = cuit_found[0] if cuit_found else None

# Print the extracted information
print(f"Bank: {bank}")
print(f"Date found: {date}")
print(f"Amount found: {amount}")
print(f"Payer name: {payer}")
print(f"CUIT found: {cuit}")
print(f"Proof number found: {proof_number}")


Bank: BBVA
Date found: 26/03/2024
Amount found: $ 122.500,00
Payer name: EL ORIGEN S.A.S
CUIT found: 30-71746255-2
Proof number found: 50076821004240326


In [58]:
#takes some files are extracted and save the data
import cv2
import pytesseract
import re
import os
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment

# Function to extract data from an image and return the extracted information
def extract_data_from_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use pytesseract to extract text from the grayscale image
    extracted_text = pytesseract.image_to_string(gray_image)

        
    lines = extracted_text.split('\n')

    #for payer name
    line_4 = lines[4]
    #removes 1st word
    line_4 = re.sub(r'^\w+:+\s+\b', '', line_4)
    # print(line_4s)
    #removes last word
    line_4 = re.sub(r'\b\s+\w+$', '', line_4)
    # print(line_4)


    #for proof number
    line_3 = lines[3]
    #removes 1st word
    # line_3 = re.sub(r'^\w+:+\s+\b', '', line_3)
    # Cuenta_Origen = r'^Cuenta Origen:\s+CC\s+\$\s'
    Cuenta_Origen = r'^Cuenta Origen:\s+CC\s+\$\s+\d{4}-\d{6}\/\d{1}\s+'
    line_3 = re.sub(Cuenta_Origen, '', line_3)
    line_3 = re.sub(r' ' , '', line_3)
    # print(line_3)

    #for amount
    line_12 = lines[12]
    # print(line_12)
    #removes 1st word
    line_12 = re.sub(r'^\w+:+\s+\b', '', line_12)
    #adds $ 
    line_12 = '$ ' + line_12
    # print(line_12)

    # Define regular expression patterns for different types of data to extract
    
    bank_pattern = 'BBVA'
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'

    amount_pattern = line_3
    payer_name_pattern = line_4
    proof_number_pattern = line_12
    
    cuit_pattern =  r'\b\d{2}-\d{8}-\d{1}\b'

    # Find all occurrences of dates, amounts, and CUIT numbers in the extracted text
    bank_found = re.findall(bank_pattern, extracted_text, re.IGNORECASE)

    dates_found = re.findall(date_pattern, extracted_text)

    amounts_found = re.findall(amount_pattern, extracted_text)
    proof_number_found = re.findall(proof_number_pattern, extracted_text)  # Use cuit_pattern
    payer_name_found = re.findall(payer_name_pattern , extracted_text)

    cuit_found = re.findall(cuit_pattern, extracted_text)  # Use cuit_pattern

    # Extract the first date, amount, and CUIT number if any are found
    date = dates_found[0] if dates_found else None

    amount = line_12
    proof_number = line_3
    payer = payer_name_found[0] if payer_name_found else None #this works

    cuit = cuit_found[0] if cuit_found else None

    # Return the extracted information as a dictionary
    return {
        'BANCO': bank,
        'FECHA': date,
        'IMPORTE': amount,
        'TITULAR': payer,
        'CUIT': cuit,
        'NRO COMPROBANTE': proof_number
    }

# Create a new Excel workbook
wb = Workbook()
ws = wb.active
ws.title = "Extracted Data"

# Add headers to the Excel sheet
headers = [ 'FECHA', 'IMPORTE', 'NRO COMPROBANTE', 'TITULAR', 'CUIT', 'BANCO']
ws.append(headers)

# Apply bold style to the header cells
bold_font = Font(bold=True)
for cell in ws[1]:
    cell.font = bold_font

# Apply padding to all cells
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Define the folder containing the images
folder_path = 'bbva_assets'

# Process each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        # Extract data from the image
        image_path = os.path.join(folder_path, filename)
        extracted_data = extract_data_from_image(image_path)

        # Add the extracted data to the Excel sheet
        row_data = [extracted_data[header] for header in headers]
        ws.append(row_data)

# Automatically adjust column widths based on content
for column_cells in ws.columns:
    max_length = 0
    for cell in column_cells:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2  # Adjust the multiplier as needed
    ws.column_dimensions[column_cells[0].column_letter].width = adjusted_width


# Print the extracted information
print(f"Bank: {bank}")
print(f"Date found: {date}")
print(f"Amount found: {amount}")
print(f"Payer name: {payer}")
print(f"CUIT found: {cuit}")
print(f"Proof number found: {proof_number}")


# Save the Excel file
wb.save('ancopatagonia_extracted_info.xlsx')

Bank: BBVA
Date found: 26/03/2024
Amount found: $ 122.500,00
Payer name: EL ORIGEN S.A.S
CUIT found: 30-71746255-2
Proof number found: 50076821004240326


In [2]:
# to convert the pdf to image expermenting
from pdf2image import convert_from_path
import os

# Path to the folder containing the PDF files
pdf_folder_path = 'bbva_assets'

# Get the list of PDF files in the folder
pdf_files = [file for file in os.listdir(pdf_folder_path) if file.endswith('.pdf')]

# Convert each PDF file to images
for pdf_file in pdf_files:
    # Construct the file paths
    pdf_file_path = os.path.join(pdf_folder_path, pdf_file)
    image_file_path = os.path.join(pdf_folder_path, os.path.splitext(pdf_file)[0] + '.png')

    # Convert PDF to list of PIL images
    images = convert_from_path(pdf_file_path)

    # Save each page of the PDF as an image file
    for i, image in enumerate(images):
        image.save(image_file_path, 'PNG')

print('PDFs converted to images successfully.')


PDFs converted to images successfully.
